In [11]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import itertools
import pickle

In [10]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 350)
(130, 298)


Inizialmente sono state eliminate le colonne che fanno riferimento ai totali delle varie zono del corpo, in quanto sono una combinazione lineare delle precedenti colonne che sono già presenti all'interno del data set.
Poi sono state eliminate le colonne a valori costanti in quanto non sono significative per la classificazione.
Sono state, infine, rimosse le colonne duplicate.
Da 367 colonne colonne iniziale, ora il data set è formato da 298 colonne.

In [12]:
def nested_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [13]:
results = nested_cv_accuracy(X)
results

{'algorythm': 'OneClassSVM',
 'best kernel': 'rbf',
 'best gamma': 1e-09,
 'best nu': 0.09999999999999999,
 'method used for model selection': 'accuracy on nested cv',
 'accuracy mean': 56.230769230769226,
 'accuracy std': 8.999999999999998,
 'precision mean': 66.06416083916083,
 'precision  std': 34.80905859076927,
 'recall mean': 28.571428571428577,
 'recall std': 21.52359881902766,
 'data set': 'no const, no equals, no tot, standard scaled'}

In [14]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.1,accuracy on nested cv,56.230769,9.0,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"


In [15]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [16]:
def nested_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(kernels, gammas, nus):
                    clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [17]:
scores_df = add_record(scores_df, nested_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"


In [18]:
def outer_cv_inner_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [19]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"


In [20]:
def outer_cv_inner_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

            clf = OneClassSVM(**best_params)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [21]:
scores_df = add_record(scores_df, outer_cv_inner_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-08,0.14,f1 on outer cv inner holdout,57.384615,9.416088,69.069894,27.997567,37.285714,29.975841,"no const, no equals, no tot, standard scaled"


In [22]:
def outer_holdout_inner_cv_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [23]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-08,0.14,f1 on outer cv inner holdout,57.384615,9.416088,69.069894,27.997567,37.285714,29.975841,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-02,0.98,accuracy on outer holdout inner cv,48.076923,8.113470,67.166667,29.758752,15.103815,7.972938,"no const, no equals, no tot, standard scaled"


In [24]:
def outer_holdout_inner_cv_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(kernels, gammas, nus):
                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['kernel'] = params[0]
                    best_params['gamma'] = params[1]
                    best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [25]:
scores_df = add_record(scores_df, outer_holdout_inner_cv_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-08,0.14,f1 on outer cv inner holdout,57.384615,9.416088,69.069894,27.997567,37.285714,29.975841,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-02,0.98,accuracy on outer holdout inner cv,48.076923,8.113470,67.166667,29.758752,15.103815,7.972938,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,rbf,1.000000e-09,0.10,f1 on outer holdout inner cv,63.461538,11.570468,69.265324,31.685051,42.093074,25.679543,"no const, no equals, no tot, standard scaled"


In [26]:
def double_holdout_accuracy(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [27]:
scores_df = add_record(scores_df, double_holdout_accuracy(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-08,0.14,f1 on outer cv inner holdout,57.384615,9.416088,69.069894,27.997567,37.285714,29.975841,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-02,0.98,accuracy on outer holdout inner cv,48.076923,8.113470,67.166667,29.758752,15.103815,7.972938,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,rbf,1.000000e-09,0.10,f1 on outer holdout inner cv,63.461538,11.570468,69.265324,31.685051,42.093074,25.679543,"no const, no equals, no tot, standard scaled"
6,OneClassSVM,rbf,1.000000e-07,0.26,accuracy on double holdout,55.769231,9.768019,86.166667,14.908052,22.269044,10.795059,"no const, no equals, no tot, standard scaled"


In [28]:
def double_holdout_f1(X):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.99, 99)
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0
        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(kernels, gammas, nus):
            clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['kernel'] = params[0]
                best_params['gamma'] = params[1]
                best_params['nu'] = params[2]

        clf = OneClassSVM(**best_params)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'OneClassSVM',
            'best kernel': best_params['kernel'],
            'best gamma': best_params['gamma'],
            'best nu': best_params['nu'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals, no tot, standard scaled'}

In [29]:
scores_df = add_record(scores_df, double_holdout_f1(X))
scores_df

,algorythm,best kernel,best gamma,best nu,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,OneClassSVM,rbf,1.000000e-09,0.10,accuracy on nested cv,56.230769,9.000000,66.064161,34.809059,28.571429,21.523599,"no const, no equals, no tot, standard scaled"
1,OneClassSVM,rbf,1.000000e-09,0.38,f1 on nested cv,58.307692,7.970359,68.422744,19.933970,45.857143,24.993469,"no const, no equals, no tot, standard scaled"
2,OneClassSVM,rbf,1.000000e-04,0.86,accuracy on outer cv inner holdout,52.692308,7.143914,65.230891,36.848251,20.285714,21.962863,"no const, no equals, no tot, standard scaled"
3,OneClassSVM,rbf,1.000000e-08,0.14,f1 on outer cv inner holdout,57.384615,9.416088,69.069894,27.997567,37.285714,29.975841,"no const, no equals, no tot, standard scaled"
4,OneClassSVM,rbf,1.000000e-02,0.98,accuracy on outer holdout inner cv,48.076923,8.113470,67.166667,29.758752,15.103815,7.972938,"no const, no equals, no tot, standard scaled"
5,OneClassSVM,rbf,1.000000e-09,0.10,f1 on outer holdout inner cv,63.461538,11.570468,69.265324,31.685051,42.093074,25.679543,"no const, no equals, no tot, standard scaled"
6,OneClassSVM,rbf,1.000000e-07,0.26,accuracy on double holdout,55.769231,9.768019,86.166667,14.908052,22.269044,10.795059,"no const, no equals, no tot, standard scaled"
7,OneClassSVM,rbf,1.000000e-09,0.68,f1 on double holdout,49.230769,10.986813,56.053552,33.014256,32.805556,27.922925,"no const, no equals, no tot, standard scaled"


In [30]:
file_path = 'svm_exp3_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)